In [1]:
from core import *
from tools import *
import numpy as np
import os
import tensorflow as tf
import keras as K

import sys
import argparse

Using TensorFlow backend.


In [2]:
# ICML_2019/DNA_CNN/Denoising

class Denoising_Process:
    def __init__(self, n, k, nb_x_classes, nb_z_classes, x, z, param_name = 'test'):
        self.n, self.k, self.x, self.z, self.nb_x_classes, self.nb_z_classes = n, k, x, z, nb_x_classes, nb_z_classes
        self.param_name = param_name
        self.raw_error = error_rate(x,z)
        self.C = make_batch(z,k)
    
    def denoise(self, pred_prob): # Denoise sequence using softmax output
        n, k, x, z = self.n, self.k, self.x, self.z
        
        """
        pred_class[0] = Say What You See(s[0]=z[i]) = -1
        pred_class[1] = Always Say 0(s[1]=0) = 0
        pred_class[2] = Always Say 1(s[2]=1) = 1
        pred_class[3] = Always Say 1(s[2]=1) = 2
        pred_class[4] = Always Say 1(s[2]=1) = 3
        """
        
        # s(z) = z
        pred_class = np.argmax(pred_prob, axis = -1) - 1
        
        # mask Say What You see
        mask = pred_class == -1
        
        # mask-> Say What You see || others-> 0,1,2,3
        x_hat = z * mask + (mask^1)*pred_class

        error = normalized_error_rate(x,x_hat,self.raw_error)
        return error, x_hat
    
    def N_DUDE(self, PI): # Denoising process
        n, k, nb_x_classes, nb_z_classes, z, param_name, C = self.n, self.k, self.nb_x_classes, self.nb_z_classes, self.z, self.param_name, self.C
        
        
        # fine-tuning the weights from ICE process
        L_new = L_NEW(PI, nb_x_classes, nb_z_classes)
        Y = make_pseudo_label(C, k, L_new)
        
        # model assign & train
        if param_name == 'NDUDE':
            model = NDUDE_CNN_model_5map(1000, nb_x_classes, nb_z_classes, k)
            hist = model.fit(C,Y,epochs=20, batch_size=2*4, verbose=1, validation_data=(C, Y))
        
        else:
            model = NDUDE_CNN_model_5map(1000, nb_x_classes, nb_z_classes, k, lr = 0.0001)
            model.load_weights("weights/"+param_name+".hd5")
            hist = model.fit(C,Y,epochs=10, batch_size=2*4, verbose=1, validation_data=(C, Y))
            
        pred_prob = model.predict(C, batch_size = 20*4, verbose = 0)
        
        # reshape the output
        N,D,_ = pred_prob.shape
        pred_prob = np.resize(pred_prob, (N*D,nb_x_classes+1))[:n]
        
        return self.denoise(pred_prob)


In [3]:
PI_true = array([ [ 0.8122,  0.0034,  0.0894,  0.0950],
                  [ 0.0096,  0.8237,  0.0808,  0.0859],
                  [ 0.1066,  0.0436,  0.7774,  0.0724],
                  [ 0.0704,  0.0690,  0.0889,  0.7717]])
x, z = load_DNA(PI_true)

try:
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--k", help="window size k", type=int)
    
    args = parser.parse_args()
    
    result_name = sys.argv[0]
    k = args.k
    n = len(x)
    
except:
    result_name = "test"
    k = 150
    n = int(1e4)
    n = len(x)

nb_x_classes, nb_z_classes = 4, 4
param_name = 'NDUDE'
x, z = x[:n], z[:n]

usage: ipykernel_launcher.py [-h] [--k K]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1002/jupyter/kernel-4b72b368-ff5a-43cb-9a46-0c6bdd8b7784.json


In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0, 1, 2, 3'

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.backend.set_session(session)

In [5]:
f = open('results/'+result_name,'a')

print("k: %d "%(k))

DE = Denoising_Process(n, k, nb_x_classes, nb_z_classes, x, z, param_name = param_name)
normalized_error, x_hat = DE.N_DUDE(PI_true)
f.write("%d %.5f\n"%(k, normalized_error))
print(normalized_error)

k: 150 
Train on 2470 samples, validate on 2470 samples
Epoch 1/20
2470/2470 [==============================] - 9s 4ms/step - loss: 0.7648 - val_loss: 0.7310
Epoch 2/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7210 - val_loss: 0.7105
Epoch 3/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7075 - val_loss: 0.7009
Epoch 4/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.7002 - val_loss: 0.6954
Epoch 5/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.6960 - val_loss: 0.6921
Epoch 6/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.6930 - val_loss: 0.6895
Epoch 7/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.6907 - val_loss: 0.6876
Epoch 8/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.6890 - val_loss: 0.6857
Epoch 9/20
2470/2470 [==============================] - 4s 2ms/step - loss: 0.6874 - val_loss: 0.6844
Epoch 10/20
2470/2470 [===